In [11]:
%load_ext autoreload
%autoreload 1

In [7]:
from pathlib import Path

import pandas as pd
import geopandas as gpd
import pystac

from kappa.paths import KappazunderPath


In [8]:
data_dir = Path('./assets/Los_6A/')
kappa_path = KappazunderPath(data_dir)

In [3]:
def get_image_df(data_dir: Path) -> gpd.GeoDataFrame:
    image_meta_path = data_dir / 'Bild-Meta/image_meta.txt'
    df = pd.read_csv(image_meta_path, sep="\s+")
    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df.x_m, df.y_m, df.z_m, crs="epsg:31256"),
        crs="epsg:31256"
    )
    return gdf

    

In [4]:
from dataclasses import dataclass
from typing import Literal
import re


def get_direction_label(
    sensor_id: str,
) -> Literal["up", "front", "right", "back", "left", "down"]:
    # Can also calculate it, given the trajectory and sensor angles
    MAPPING = {
        "0": "up",
        "1": "front",
        "2": "right",
        "3": "back",
        "4": "left",
        "5": "down",
    }
    return MAPPING[sensor_id[-1]]


In [5]:
from datetime import datetime, timezone
import gpsdatetime as gpst
from pystac.extensions.projection import ProjectionExtension
from tqdm import tqdm
from pprint import pprint
from astropy.time import Time




@dataclass
class TrajectoryInfo:
    gps_week: int
    epsg: str
    
def gps_time_to_datetime(gps_week: int, gps_seconds: float) -> datetime:
    SECONDS_IN_WEEK = 604800
    time = Time(SECONDS_IN_WEEK * gps_week + gps_seconds, format='gps')
    return time.datetime.replace(tzinfo=timezone.utc)
    


def main(base_dir: Path):
    assert base_dir.exists()
    kappa_path = KappazunderPath(base_dir)
        
    spatial_extent = pystac.SpatialExtent([[-180.0, -90.0, 180.0, 90.0]])
    temporal_extent = pystac.TemporalExtent([[datetime(2013, 6, 1), None]])
    collection_extent = pystac.Extent(spatial_extent, temporal_extent)

    collection = pystac.Collection(
        id='kappa_collection',
        title='Vienna mobile mapping campaign 2020',
        description="Kappazunder 2020 data extract",
        extent=collection_extent
    )

    df = pd.read_csv(kappa_path.image_metadata, sep='\s+')

    for _, group in df.groupby('image_id'):
        collection.add_item(create_image_item(kappa_path, group))
        
    collection.update_extent_from_items()
    
    collection.normalize_hrefs(str((kappa_path.base_dir / 'stac').absolute()))

    collection.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)

def get_trajectory_info(kappa_path: KappazunderPath, trajectory_id: str) -> TrajectoryInfo:
    trajectory_path = kappa_path.trajectory(trajectory_id)
    match = re.search(r'trajectory_\d+_(\d+)_(\d+)', trajectory_path.name)
        
    info = TrajectoryInfo(
        gps_week=int(match.group(1)),
        epsg=int(match.group(2))
    )
    
    return info

def create_image_item(kappa_path: KappazunderPath, image_group: pd.DataFrame) -> pystac.Item:
    image = image_group.iloc[0]
    
    trajectorie_info = get_trajectory_info(kappa_path, image.trajectory_id)
    gps_epoch = image.epoch_s
    
    item = pystac.Item(
        id=str(image.image_id),
        bbox=[image.x_m, image.y_m, image.x_m, image.y_m],
        geometry={
            "type": "Point",
            "coordinates": [image.x_m, image.y_m, image.z_m],
        },
        datetime=gps_time_to_datetime(trajectorie_info.gps_week, gps_epoch),
        properties={
            "trajectory_id": image.trajectory_id,
            "gps_week": trajectorie_info.gps_week,
            "gps_epoch_s": gps_epoch,
        }
    )
    proj_ext = ProjectionExtension.ext(item, add_if_missing=True)
    proj_ext.epsg = trajectorie_info.epsg

    for image in tqdm(image_group.itertuples()):
        direction = get_direction_label(str(image.sensor_id))
        item.add_asset(
            direction,
            pystac.Asset(
                href=kappa_path.raw_image(trajectory_id=image.trajectory_id, sensor_id=image.sensor_id, image_name=image.image_name),
                title=f"{direction.capitalize()} photo",
                media_type=pystac.MediaType.JPEG,
                extra_fields={
                    'rx_rad': image.rx_rad,
                    'ry_rad': image.ry_rad,
                    'rz_rad': image.rz_rad,
                }
        ))
    # item.validate()
    return item

In [6]:
import json
from owslib.wfs import WebFeatureService
from requests import Request

WFS_URL = "https://data.wien.gv.at/daten/geo?version=1.1.0&service=WFS" # &request=GetCapabilities


In [7]:
wfs = WebFeatureService(url=WFS_URL, version='1.1.0', timeout=60)

In [8]:
[name for name in wfs.contents if name.startswith('ogdwien:KAPPA')]

['ogdwien:KAPPAZUNDERIMAGEPOGD',
 'ogdwien:KAPPAZUNDERLIDARFOGD',
 'ogdwien:KAPPAZUNDERPANOPOGD']

In [9]:
wfs.get_schema('ogdwien:KAPPAZUNDERIMAGEPOGD')

{'properties': {'OBJECTID': 'decimal',
  'LOSID': 'string',
  'TRAJECTORYID': 'decimal',
  'EPOCH': 'dateTime',
  'IMAGE_NAME': 'string'},
 'required': ['OBJECTID'],
 'geometry': 'GeometryCollection',
 'geometry_column': 'SHAPE'}

In [10]:
wfs.get_schema('ogdwien:KAPPAZUNDERPANOPOGD')

{'properties': {'OBJECTID': 'decimal',
  'TRAJECTORYID': 'decimal',
  'EPOCH_S': 'decimal',
  'IMAGE_NAME': 'string',
  'LOSID': 'string',
  'GPS_WEEK': 'decimal'},
 'required': ['OBJECTID'],
 'geometry': 'GeometryCollection',
 'geometry_column': 'SHAPE'}

In [11]:
wfs.get_schema('ogdwien:KAPPAZUNDERLIDARFOGD')

{'properties': {'OBJECTID': 'decimal',
  'TRAJECTORYID': 'decimal',
  'EPOCH_START_S': 'decimal',
  'SCANDATA_NAME': 'string',
  'LOSID': 'string',
  'GPS_WEEK': 'decimal'},
 'required': ['OBJECTID'],
 'geometry': 'GeometryCollection',
 'geometry_column': 'SHAPE'}

In [12]:
def select(d, keys):
    return {
        k: v
        for k, v in d.items()
        if k in keys
    }
    
def omit(d, keys):
    return {
        k: v
        for k, v in d.items()
        if k not in keys
    }
    


In [13]:
omit(data, ['features'])

NameError: name 'data' is not defined

In [14]:
omit(data, ['features'])

NameError: name 'data' is not defined

In [15]:
# PANO_FEATURE = data['features'][0]

In [16]:
IMAGE_FEATURE = data['features'][0]

NameError: name 'data' is not defined

In [17]:
PANO_FEATURE

NameError: name 'PANO_FEATURE' is not defined

In [18]:
IMAGE_FEATURE

NameError: name 'IMAGE_FEATURE' is not defined

In [19]:
from math import ceil
import orjson as json
from requests import Request


def get_all_features(wfs: WebFeatureService, layer_name: str, batch_size: int = 50000) -> gpd.GeoDataFrame:
    resp = wfs.getfeature(layer_name, outputFormat='json', maxfeatures=1)
    data = json.loads(resp.read())
    result = gpd.GeoDataFrame()
    total = data['totalFeatures']
    for offset in tqdm(range(0, total, batch_size), desc='Fetching data...', total=ceil(total / batch_size), unit="batches"):
        params = dict(service='WFS', version="1.1.0", request='GetFeature', typeName=layer_name, outputFormat='json', maxFeatures=batch_size, startIndex=offset, sortby="OBJECTID")
        wfs_request_url = Request('GET', WFS_URL, params=params).prepare().url

        gdf = gpd.read_file(wfs_request_url).set_crs('epsg:31256')
        result = pd.concat([result, gdf])
    return result
    

In [457]:
all_images = get_all_features(wfs, 'ogdwien:KAPPAZUNDERIMAGEPOGD')

Fetching data...:   4%|████▊                                                                                                                                | 1/28 [00:06<03:04,  6.84s/it]

(50000, 7)


Fetching data...:   7%|█████████▌                                                                                                                           | 2/28 [00:14<03:10,  7.31s/it]

(50000, 7)


KeyboardInterrupt: 

In [11]:
all_images.shape

NameError: name 'all_images' is not defined

In [397]:
from shapely import force_2d, wkb

# _drop_z = lambda geom: wkb.loads(wkb.dumps(geom, output_dimension=2))
gdf.geometry = gdf.geometry.map(force_2d)


In [395]:
type(gdf.geometry.iloc[0])

shapely.geometry.point.Point

In [399]:
gdf.head(5)

,id,OBJECTID,LOSID,TRAJECTORYID,EPOCH,IMAGE_NAME,geometry
0,KAPPAZUNDERIMAGEPOGD.2742943,2742943,1,17072,2020-07-05 04:22:20+00:00,WE1LBS98.jpg,POINT (6512.896 343098.513)
1,KAPPAZUNDERIMAGEPOGD.2742944,2742944,1,17072,2020-07-05 04:22:25+00:00,WE1LBS99.jpg,POINT (6513.052 343095.874)
2,KAPPAZUNDERIMAGEPOGD.2742945,2742945,1,17072,2020-07-05 04:22:27+00:00,WE1LBS9A.jpg,POINT (6512.606 343092.819)
3,KAPPAZUNDERIMAGEPOGD.2742946,2742946,1,17072,2020-07-05 04:22:30+00:00,WE1LBS9B.jpg,POINT (6512.542 343089.786)
4,KAPPAZUNDERIMAGEPOGD.2742947,2742947,1,17072,2020-07-05 04:22:57+00:00,WE1LBS9D.jpg,POINT (6513.972 343085.63)


In [398]:
from lonboard import viz

viz(gdf.to_crs(4326))

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [368]:
layer_name = 'ogdwien:KAPPAZUNDERIMAGEPOGD'
batch_size: int = 100000
offset = 0

resp = wfs.getfeature(layer_name, outputFormat='json', maxfeatures = batch_size, startindex=offset)
gdf = gpd.read_file(resp).set_crs('epsg:31256')


In [344]:
meta = wfs.contents['ogdwien:KAPPAZUNDERIMAGEPOGD']

In [517]:
kappa_path = KappazunderPath(Path('/Users/oleksii.vykaliuk/Downloads/kappa_raw_around_stromsalon'))


In [522]:
df = pd.read_csv(kappa_path.image_metadata, sep="\s+")

rename_map = {
    'trajectory_id': 'TRAJECTORYID',
    'image_name': 'IMAGE_NAME'
}

df = df[df.sensor_id % 10 == 0].rename(columns=rename_map)[[*rename_map.values(), 'rx_rad', 'ry_rad', 'rz_rad']]

new_all = all_images.merge(df, on=['TRAJECTORYID', 'IMAGE_NAME'], how='left')

In [524]:
new_all.head()

,id,OBJECTID,LOSID,TRAJECTORYID,EPOCH,IMAGE_NAME,geometry,rx_rad,ry_rad,rz_rad
0,KAPPAZUNDERIMAGEPOGD.2742943,2742943,1,17072,2020-07-05 04:22:20+00:00,WE1LBS98.jpg,POINT (16.41979 48.22624),NaN,NaN,NaN
1,KAPPAZUNDERIMAGEPOGD.2742944,2742944,1,17072,2020-07-05 04:22:25+00:00,WE1LBS99.jpg,POINT (16.41979 48.22622),NaN,NaN,NaN
2,KAPPAZUNDERIMAGEPOGD.2742945,2742945,1,17072,2020-07-05 04:22:27+00:00,WE1LBS9A.jpg,POINT (16.41979 48.22619),NaN,NaN,NaN
3,KAPPAZUNDERIMAGEPOGD.2742946,2742946,1,17072,2020-07-05 04:22:30+00:00,WE1LBS9B.jpg,POINT (16.41979 48.22617),NaN,NaN,NaN
4,KAPPAZUNDERIMAGEPOGD.2742947,2742947,1,17072,2020-07-05 04:22:57+00:00,WE1LBS9D.jpg,POINT (16.41981 48.22613),NaN,NaN,NaN


In [531]:
new_all['has_images'] = new_all.rx_rad.notnull()

In [536]:
new_all.to_file('./output/json/images_merged.geojson', driver='geojson')

In [ ]:
ids = ['image_name', 'trajectory_id']
fields_to_save = ['rx_rad', 'ry_rad', 'rz_rad']

In [504]:
all_images = gpd.read_file('./output/json/all_images.geojson', driver='geojson')

Warning 6: driver GeoJSON does not support open option DRIVER


In [505]:
all_images.head()

,id,OBJECTID,LOSID,TRAJECTORYID,EPOCH,IMAGE_NAME,geometry
0,KAPPAZUNDERIMAGEPOGD.2742943,2742943,1,17072,2020-07-05 04:22:20+00:00,WE1LBS98.jpg,POINT (16.41979 48.22624)
1,KAPPAZUNDERIMAGEPOGD.2742944,2742944,1,17072,2020-07-05 04:22:25+00:00,WE1LBS99.jpg,POINT (16.41979 48.22622)
2,KAPPAZUNDERIMAGEPOGD.2742945,2742945,1,17072,2020-07-05 04:22:27+00:00,WE1LBS9A.jpg,POINT (16.41979 48.22619)
3,KAPPAZUNDERIMAGEPOGD.2742946,2742946,1,17072,2020-07-05 04:22:30+00:00,WE1LBS9B.jpg,POINT (16.41979 48.22617)
4,KAPPAZUNDERIMAGEPOGD.2742947,2742947,1,17072,2020-07-05 04:22:57+00:00,WE1LBS9D.jpg,POINT (16.41981 48.22613)


In [538]:
tmp = all_images.groupby('TRAJECTORYID').nunique()
tmp[tmp.LOSID > 1]

,id,OBJECTID,LOSID,EPOCH,IMAGE_NAME,geometry
TRAJECTORYID,,,,,,
15714,9843,9843,2,3734,9843,9843
15730,8862,8862,2,3156,8862,8862
15752,2809,2809,2,1374,2809,2809
15754,1811,1811,2,913,1811,1811
15760,6410,6410,2,3334,6410,6410
15766,5513,5513,2,2556,5513,5513
15767,9680,9680,2,4067,9680,9680
15769,2475,2475,2,642,2475,2475
15775,7663,7663,2,3333,7663,7663


In [24]:
points_clouds = get_all_features(wfs, 'ogdwien:KAPPAZUNDERLIDARFOGD', batch_size=2500)

Fetching data...: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [01:33<00:00, 10.34s/batches]


In [20]:
layer_name = 'ogdwien:KAPPAZUNDERLIDARFOGD'
resp = wfs.getfeature(layer_name, outputFormat='json', maxfeatures = 5000, startindex=0, sortby='OBJECTID')
gdf = gpd.read_file(resp).set_crs('epsg:31256')


In [32]:
points_clouds.to_crs(4326).to_file('./output/json/lidar.geojson', driver='geojson')

In [21]:
gdf.shape

(5000, 8)

In [22]:
gdf.head(3)

,id,OBJECTID,TRAJECTORYID,EPOCH_START_S,SCANDATA_NAME,LOSID,GPS_WEEK,geometry
0,KAPPAZUNDERLIDARFOGD.2,2,16848,218580.000225,scandata_19.laz,2,2112,"POLYGON Z ((5749.497 348701.997 0, 5749.305 34..."
1,KAPPAZUNDERLIDARFOGD.3,3,16848,218940.000257,scandata_25.laz,2,2112,"POLYGON Z ((5005.933 348644.76 0, 5006.021 348..."
2,KAPPAZUNDERLIDARFOGD.4,4,16848,221280.000234,scandata_61.laz,2,2112,"POLYGON Z ((7414.219 350043.791 0, 7412.32 350..."


In [12]:
kappa_path.get_all_scans()

In [5]:
kappa_path.scan_data_dir.exists()

True

In [27]:
laz_paths = list(kappa_path.scan_data_dir.glob('**/*[0-9].laz'))

In [31]:
laz_file = laz_paths[0]
laz_file

PosixPath('assets/Los_6A/Scan-Punktwolken/Trajektorie_16882/Sensor_11003/scandata_16.laz')

In [29]:
import laspy


In [33]:
# las = laspy.read(str(laz_file))
with laspy.open(laz_file) as f:
    print(f.header.point_count)
    

23118990


In [36]:
f.header.file_source_id

0

In [37]:
f.header.__dict__

{'file_source_id': 0,
 'global_encoding': <laspy.header.GlobalEncoding at 0x10ddc9060>,
 'uuid': UUID('00000000-0000-0000-0000-000000000000'),
 '_version': Version(major=1, minor=4),
 'system_identifier': 'MERGE&FILTER',
 'generating_software': 'make_ray_cells',
 '_point_format': <PointFormat(7, 0 bytes of extra dims)>,
 'creation_date': datetime.date(2022, 3, 1),
 'point_count': 23118990,
 'scales': array([0.001, 0.001, 0.001]),
 'offsets': array([1.61431184e+03, 3.39951381e+05, 3.59473562e+01]),
 'maxs': array([1.67834284e+03, 3.39967507e+05, 6.44313562e+01]),
 'mins': array([1.57758884e+03, 3.39726904e+05, 2.15293562e+01]),
 'number_of_points_by_return': array([23118990,        0,        0,        0,        0,        0,
               0,        0,        0,        0,        0,        0,
               0,        0,        0], dtype=uint32),
 '_vlrs': [<laspy.vlrs.known.WktCoordinateSystemVlr object at 0x10e159180>, <laspy.vlrs.known.LasZipVlr object at 0x10e147df0>],
 'extra_header_b

In [1]:
import py3dtiles

In [2]:
from py3dtiles.convert import convert

In [3]:
convert?

Signature:
convert(
    files: Union[list[Union[str, pathlib.Path]], str, pathlib.Path],
    outfolder: Union[str, pathlib.Path] = './3dtiles',
    overwrite: bool = False,
    jobs: int = 8,
    cache_size: int = 1638,
    crs_out: Optional[pyproj.crs.crs.CRS] = None,
    crs_in: Optional[pyproj.crs.crs.CRS] = None,
    force_crs_in: bool = False,
    pyproj_always_xy: bool = False,
    benchmark: Optional[str] = None,
    rgb: bool = True,
    classification: bool = True,
    intensity: bool = True,
    color_scale: Optional[float] = None,
    use_process_pool: bool = True,
    verbose: int = False,
) -> None
Docstring:
Convert the input dataset into 3dtiles. For the argument list and their effects, please see :py:class:`.Converter`.

:param files: Filenames to process. The file must use the .las, .laz, .xyz or .ply format.
:param outfolder: The folder where the resulting tileset will be written.
:param overwrite: Overwrite the ouput folder if it already exists.
:param jobs: The numb